In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

import sys
%matplotlib inline

sys.path.append('C:\\Users\\jander33\\Desktop\\projects\\code\\quickpipeline')
sys.path.append(r'C:\Miniconda3\envs\tf\Lib\site-packages')
#print(sys.path)
from quickpipeline import QuickPipeline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Import dfs

In [3]:
cat_df = pd.read_csv(r'..\data\processed3\cat_df.csv')
num_df = pd.read_csv(r'..\data\processed3\num_df.csv')
text_df = pd.read_csv(r'..\data\processed3\text_df.csv')
res_out = pd.read_csv(r'..\data\processed3\outliers_response.csv')


In [4]:
data = pd.concat([num_df, cat_df, text_df, res_out], axis=1)


In [9]:
X = data[data['outliers'] == 1]
X.drop(['outliers'], axis=1, inplace=True)
X.reset_index(level=0, drop=True, inplace=True)
X.shape

C:\Users\jander33\AppData\Local\Continuum\miniconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(26699, 112)

In [10]:
X.head(2)

,TRV.Price,tot_bld_annl_rev_amt,fnl_prc_pt_fct,X,terr_pctl_weather,terr_pctl_bldgfire,terr_pctl_liab,terr_pctl_bldgcrime,prcnt_blg_tiv,terr_pctl_cntscrime,terr_pctl_water,rrv_rt_fct,tot_ref_rule_cnt,terr_pctl_cntsfire,nbr_comp,mnln_cd,SUM_BLD_SQFTAGE_bkt_medium,earlyshop_mth_bkt_0,Segment_RESTAURANTS,Segment_BUILDING,FirstQuoteYear_2017,UWC_KANSAS CITY,Region_NORTHEAST,Segment_CONTRACTORS,FirstEffYear_2018,earlyshop_mth_bkt_1,csf,UWC_METRO,LOC_BLD_AGE_bkt_lowest,State_VA,SPRNKL_IND_bkt,LOC_BLD_AGE_bkt_high,FirstEffYear_2017,div_none,coast_ind_Y,LOC_BLD_AGE_bkt_low,Segment_OFFICE,State_CT,Segment_BUSINESS,State_UT,Region_WESTERN,SUM_BLD_SQFTAGE_bkt_high,Segment_STORE,State_TN,State_NV,market_sh_low,FirstQuoteYear_2016,SUM_BLD_SQFTAGE_bkt_low,Region_MID-ATLANTIC,UWC_DENVER,UWC_CHARLOTTE,State_AZ,Segment_APARTMENTS,Segment_CONDOMINIUMS,State_MS,LOC_BLD_AGE_bkt_highest,Region_CENTRAL,State_NC,market_sh_med,Segment_TECH OFFICE,State_NM,FirstQuoteYear_2018,Region_SOUTH CENTRAL,LOC_BLD_AGE_bkt_medium,State_CA,UWC_HARTFORD,State_TX,earlyshop_mth_bkt_3,UWC_ATLANTA,State_DC,div_low,State_NY,UWC_HUNT VALLEY,div_med,coast_ind_N,merge_freq,w2v,program_freq,zip_freq,comp_freq,program_rev_CONSULTANTS - MANAGEMENT,program_rev_OFC PAC LAWYERS,Comp_rev_Hartford,program_rev_STORE PAC CONVENIENCE STORE W,program_rev_BLDG PAC APART INCIDENTAL MERC,program_rev_BLDG PAC GAS STATIONS - LESSOR,program_rev_BUS PAC NAIL SALONS,program_rev_GARAGE PAC GENERAL REPAIRS WIT,program_rev_JANITORIAL SERVICES,program_rev_APART PAC BLDG 13-24 UNITS,program_rev_BLDG PAC MERCANTILE OTHER,program_rev_STORE PAC LIQUOR STORES,program_rev_RESTAURANT PIZZA-FAMILY STYLE,program_rev_less common,Comp_rev_Chubb,program_rev_RESTAURANT ALL OTHER - FAMILY,Comp_rev_Hanover,program_rev_BUS PAC DAY SPA,program_rev_APART PAC BLDG 1-4 UNITS,program_rev_BLDG PAC MERCANTILE SHOP CENTE,program_rev_BUS PAC COIN OPERATED LAUNDRIE,program_rev_OFC PAC DENTISTS,program_rev_OFC PAC ACTG BKNG FIRMS,program_rev_STORE PAC GROCERY STORE (UP TO,Comp_rev_Guard Insurance (Berkshire Hathaway),program_rev_RESTAURANT OTHER ETHIC CUISINE,program_rev_CON PAC PAINT- INTERIOR & PAPE,Comp_rev_CNA,program_rev_RESTAURANT ORIENTAL TAKE-OUT,program_rev_CONDO PAC BLDG 1-4 UNITS,TRV.Win,Competitor.Premium
0,0.318529,0.708039,0.463028,-1.736114,1.754690,-1.312449,1.428424,0.873420,-0.911629,-0.810463,-0.302422,-2.363606,-0.493695,-0.193038,-0.337073,1,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,-0.148597,-1.142856,0.569885,0.765696,-1.145432,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2355.0
1,0.796515,-0.234062,-0.810597,-1.736015,-1.531996,1.476947,0.816578,1.466077,1.212265,1.328579,1.115625,-0.067562,0.678700,0.750504,4.169789,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,-0.594176,-0.285041,1.230415,-0.150713,1.290033,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4100.0


In [11]:
y = X[['TRV.Win']]
X.drop(['TRV.Win', 'Competitor.Premium'], axis=1, inplace=True)
y['TRV.Win'].value_counts()

C:\Users\jander33\AppData\Local\Continuum\miniconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    18672
1     8027
Name: TRV.Win, dtype: int64

In [12]:
X.head()

,TRV.Price,tot_bld_annl_rev_amt,fnl_prc_pt_fct,X,terr_pctl_weather,terr_pctl_bldgfire,terr_pctl_liab,terr_pctl_bldgcrime,prcnt_blg_tiv,terr_pctl_cntscrime,terr_pctl_water,rrv_rt_fct,tot_ref_rule_cnt,terr_pctl_cntsfire,nbr_comp,mnln_cd,SUM_BLD_SQFTAGE_bkt_medium,earlyshop_mth_bkt_0,Segment_RESTAURANTS,Segment_BUILDING,FirstQuoteYear_2017,UWC_KANSAS CITY,Region_NORTHEAST,Segment_CONTRACTORS,FirstEffYear_2018,earlyshop_mth_bkt_1,csf,UWC_METRO,LOC_BLD_AGE_bkt_lowest,State_VA,SPRNKL_IND_bkt,LOC_BLD_AGE_bkt_high,FirstEffYear_2017,div_none,coast_ind_Y,LOC_BLD_AGE_bkt_low,Segment_OFFICE,State_CT,Segment_BUSINESS,State_UT,Region_WESTERN,SUM_BLD_SQFTAGE_bkt_high,Segment_STORE,State_TN,State_NV,market_sh_low,FirstQuoteYear_2016,SUM_BLD_SQFTAGE_bkt_low,Region_MID-ATLANTIC,UWC_DENVER,UWC_CHARLOTTE,State_AZ,Segment_APARTMENTS,Segment_CONDOMINIUMS,State_MS,LOC_BLD_AGE_bkt_highest,Region_CENTRAL,State_NC,market_sh_med,Segment_TECH OFFICE,State_NM,FirstQuoteYear_2018,Region_SOUTH CENTRAL,LOC_BLD_AGE_bkt_medium,State_CA,UWC_HARTFORD,State_TX,earlyshop_mth_bkt_3,UWC_ATLANTA,State_DC,div_low,State_NY,UWC_HUNT VALLEY,div_med,coast_ind_N,merge_freq,w2v,program_freq,zip_freq,comp_freq,program_rev_CONSULTANTS - MANAGEMENT,program_rev_OFC PAC LAWYERS,Comp_rev_Hartford,program_rev_STORE PAC CONVENIENCE STORE W,program_rev_BLDG PAC APART INCIDENTAL MERC,program_rev_BLDG PAC GAS STATIONS - LESSOR,program_rev_BUS PAC NAIL SALONS,program_rev_GARAGE PAC GENERAL REPAIRS WIT,program_rev_JANITORIAL SERVICES,program_rev_APART PAC BLDG 13-24 UNITS,program_rev_BLDG PAC MERCANTILE OTHER,program_rev_STORE PAC LIQUOR STORES,program_rev_RESTAURANT PIZZA-FAMILY STYLE,program_rev_less common,Comp_rev_Chubb,program_rev_RESTAURANT ALL OTHER - FAMILY,Comp_rev_Hanover,program_rev_BUS PAC DAY SPA,program_rev_APART PAC BLDG 1-4 UNITS,program_rev_BLDG PAC MERCANTILE SHOP CENTE,program_rev_BUS PAC COIN OPERATED LAUNDRIE,program_rev_OFC PAC DENTISTS,program_rev_OFC PAC ACTG BKNG FIRMS,program_rev_STORE PAC GROCERY STORE (UP TO,Comp_rev_Guard Insurance (Berkshire Hathaway),program_rev_RESTAURANT OTHER ETHIC CUISINE,program_rev_CON PAC PAINT- INTERIOR & PAPE,Comp_rev_CNA,program_rev_RESTAURANT ORIENTAL TAKE-OUT,program_rev_CONDO PAC BLDG 1-4 UNITS
0,0.318529,0.708039,0.463028,-1.736114,1.754690,-1.312449,1.428424,0.873420,-0.911629,-0.810463,-0.302422,-2.363606,-0.493695,-0.193038,-0.337073,1,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,-0.148597,-1.142856,0.569885,0.765696,-1.145432,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.796515,-0.234062,-0.810597,-1.736015,-1.531996,1.476947,0.816578,1.466077,1.212265,1.328579,1.115625,-0.067562,0.678700,0.750504,4.169789,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,-0.594176,-0.285041,1.230415,-0.150713,1.290033,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.280883,-0.234062,-1.131689,-1.735620,-1.531996,-1.765727,-1.188917,-1.706381,0.223076,-1.792319,1.080174,-0.067562,-0.493695,-0.193038,-0.337073,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,-0.294798,-0.505919,0.492350,-0.303448,0.441252,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,-1.297637,-0.668225,0.252468,-1.735521,-1.052688,-0.057221,0.068767,0.420212,-0.911629,0.942850,1.541039,-0.067562,-0.493695,0.715558,-0.337073,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-0.578262,0.303904,0.057951,-0.761653,-0.898773,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.810146,-0.079438,-0.389553,-1.735422,-0.813034,1.337478,0.544647,1.326629,1.212265,1.012983,1.044723,-0.067562,1.851095,-0.018308,-0.337073,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-0.216

### Base model -> shuffled

In [13]:
from xgboost import XGBClassifier
from xgboost import plot_importance, plot_tree, to_graphviz

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20)


model = XGBClassifier()

model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_logloss = model.predict_proba(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print('log-loss: ', log_loss(y_test, y_logloss))


C:\Users\jander33\AppData\Local\Continuum\miniconda3\envs\DL\lib\site-packages\sklearn\preprocessing\label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jander33\AppData\Local\Continuum\miniconda3\envs\DL\lib\site-packages\sklearn\preprocessing\label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[3518  264]
 [ 998  560]]
             precision    recall  f1-score   support

          0       0.78      0.93      0.85      3782
          1       0.68      0.36      0.47      1558

avg / total       0.75      0.76      0.74      5340

log-loss:  0.481041155709


#### This is already somewhat optimized

In [ ]:
XGBClassifier()

XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, 
              objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, 
              gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, 
              colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, 
              base_score=0.5, score=0.5, random_state=0, seed=None, missing=None,

In [15]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    
    'learning_rate': [0.1],
    'max_depth': [5],
    'min_child_weight': [20],
    'gamma': [0.1],
    'colsample_bytree': [0.7],
    'reg_alpha': [0.1],
    'reg_lambda': [0.1],
    'max_delta_step': [1],
    'n_estimators': [100],
    
    'seed': [81]
}
# Create a based model
model = XGBClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)


In [18]:
grid_search.fit(X, np.ravel(y))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   22.1s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'seed': [81], 'max_delta_step': [1], 'n_estimators': [100], 'colsample_bytree': [0.7], 'max_depth': [5], 'reg_lambda': [0.1], 'reg_alpha': [0.1], 'gamma': [0.1], 'learning_rate': [0.1], 'min_child_weight': [20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [19]:
grid_search.best_params_

{'colsample_bytree': 0.7,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_delta_step': 1,
 'max_depth': 5,
 'min_child_weight': 20,
 'n_estimators': 100,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1,
 'seed': 81}

In [20]:
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
y_logloss = best_grid.predict_proba(X_test)

In [21]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print('log-loss: ', log_loss(y_test, y_logloss))

[[3545  237]
 [ 872  686]]
             precision    recall  f1-score   support

          0       0.80      0.94      0.86      3782
          1       0.74      0.44      0.55      1558

avg / total       0.79      0.79      0.77      5340

log-loss:  0.440226031947
